<a href="https://colab.research.google.com/github/alucinationx/Cripto_Modelling/blob/main/predictionethprofeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install yfinance
!pip install --no-cache-dir -I pystan==2.19.1.1

     |████████████████████████████████| 67.3MB 1.6MB/s 
     |████████████████████████████████| 15.3MB 35.3MB/s 
     |████████████████████████████████| 2.0MB 30.3MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [8]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [91]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download('ETH-USD',start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-06-05,"$2,691.62","$2,817.48","$2,558.23","$2,630.58","$2,630.58",30496672724
2021-06-06,"$2,629.75","$2,743.44","$2,616.16","$2,715.09","$2,715.09",25311639414
2021-06-07,"$2,713.05","$2,845.19","$2,584.00","$2,590.26","$2,590.26",30600111277
2021-06-08,"$2,594.60","$2,620.85","$2,315.55","$2,517.44","$2,517.44",41909736778
2021-06-09,"$2,520.21","$2,579.36","$2,412.20","$2,464.73","$2,464.73",36131217408


In [92]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1983 entries, 2016-01-01 to 2021-06-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1983 non-null   float64
 1   High       1983 non-null   float64
 2   Low        1983 non-null   float64
 3   Close      1983 non-null   float64
 4   Adj Close  1983 non-null   float64
 5   Volume     1983 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 108.4 KB


In [93]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [94]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [95]:
eth_df.reset_index(inplace=True)
eth_df.columns 

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [96]:
df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [97]:
df.tail()

,ds,y
1978,2021-06-05,"$2,691.62"
1979,2021-06-06,"$2,629.75"
1980,2021-06-07,"$2,713.05"
1981,2021-06-08,"$2,594.60"
1982,2021-06-09,"$2,520.21"


In [98]:
# plot the open price
x = df["ds"]
y = df["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

In [99]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [100]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2343,2022-06-05
2344,2022-06-06
2345,2022-06-07
2346,2022-06-08
2347,2022-06-09


In [101]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2343,2022-06-05,"$4,683.30","$2,964.16","$6,339.32"
2344,2022-06-06,"$4,697.44","$2,993.30","$6,374.09"
2345,2022-06-07,"$4,786.08","$3,017.46","$6,503.69"
2346,2022-06-08,"$4,841.13","$3,125.55","$6,612.94"
2347,2022-06-09,"$4,843.71","$3,107.63","$6,600.73"


In [102]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()

2601.056371166874

In [103]:
plot_plotly(m, forecast)

In [77]:
plot_plotly(m, forecast)

In [38]:
plot_components_plotly(m, forecast)